**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [3]:
!pip install keras-flops

In [4]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
from keras.utils import np_utils

In [5]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [6]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [7]:
# Преобразуйте метки классов в one_hot формат
y_train = np_utils.to_categorical(y_train, CLASSES)
y_val = np_utils.to_categorical(y_val, CLASSES)

In [8]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [9]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    
    tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=(2,2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(16, kernel_size=(3,3),strides=(2,2),padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

2023-01-25 18:47:04.583794: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:04.594486: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:04.595254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:04.597062: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [10]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 18:47:06.500304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:06.500631: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 18:47:06.500792: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 18:47:06.501199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:06.501575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 18:47:06.501928: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

batch_normalization_2/batchnorm/mul (512/512 flops)
  sequential/batch_normalization_2/batchnorm/add_1 (512/512 flops)
  sequential/batch_normalization_2/batchnorm/add (512/512 flops)
  sequential/dense/BiasAdd (512/512 flops)
  sequential/activation_3/Softmax (500/500 flops)
  sequential/conv2d_1/BiasAdd (256/256 flops)
  sequential/dense_1/BiasAdd (100/100 flops)

======================End of Report==========================


In [11]:
# вывод краткой информации о модели
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 16)        448       
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 16)        64        
_________________________________________________________________
activation (Activation)      (None, 16, 16, 16)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 16)          0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 4, 16)          2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 16)          6

In [12]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [13]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=40
)

2023-01-25 18:47:07.281941: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40


2023-01-25 18:47:08.222974: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


391/391 [==============================] - 4s 6ms/step - loss: 3.6043 - accuracy: 0.1638 - val_loss: 3.2702 - val_accuracy: 0.2303
Epoch 2/40
391/391 [==============================] - 2s 5ms/step - loss: 3.0195 - accuracy: 0.2554 - val_loss: 2.9336 - val_accuracy: 0.2792
Epoch 3/40
391/391 [==============================] - 2s 4ms/step - loss: 2.7848 - accuracy: 0.3009 - val_loss: 2.7385 - val_accuracy: 0.3182
Epoch 4/40
391/391 [==============================] - 2s 5ms/step - loss: 2.6295 - accuracy: 0.3306 - val_loss: 2.6419 - val_accuracy: 0.3414
Epoch 5/40
391/391 [==============================] - 2s 5ms/step - loss: 2.5016 - accuracy: 0.3576 - val_loss: 2.6016 - val_accuracy: 0.3540
Epoch 6/40
391/391 [==============================] - 2s 4ms/step - loss: 2.4101 - accuracy: 0.3757 - val_loss: 2.4746 - val_accuracy: 0.3762
Epoch 7/40
391/391 [==============================] - 2s 5ms/step - loss: 2.3254 - accuracy: 0.3951 - val_loss: 2.4676 - val_accuracy: 0.3771
Epoch 8/40
391/39